# Use Model Context Protocol (MCP) as tools with Strands Agent

## Overview
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands AI SDK integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. The Strands Agent SDK includes built-in support for connecting to MCP servers and using their tools.

In this example we will show you how to use MCP tools on your Strands Agent. We will use the [AWS Documentation MCP server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/) which provides tools to access AWS documentation, search for content, and get recommendations. This MCP server has 3 main features:

- **Read Documentation**: Fetch and convert AWS documentation pages to markdown format
- **Search Documentation**: Search AWS documentation using the official search API
- **Recommendations**: Get content recommendations for AWS documentation pages



## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |MCP Tools                                          |
|Agent Structure     |Single agent architecture                          |

</div>

## Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## Key Features
* **Single agent architecture**: this example creates a single agent that interacts with MCP tools
* **MCP tools**: Integration of MCP tools with your agent

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock

Let's now install the requirement packages for our Strands Agent agent

In [1]:
# installing pre-requisites
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 84.6 MB/s eta 0:00:0000:0100:01


### Importing dependency packages

Now let's import the dependency packages

In [2]:
import threading
import time
from datetime import timedelta

from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient

### Connect to MCP server using stdio transport

[Transports](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports) in MCP provide the foundations for communication between clients and servers. It handles the underlying mechanics of how messages are sent and received. At the moment there are three standards transport implementations built-in in MCP:

- **Standard Input/Output (stdio)**: enables communication through standard input and output streams. It is particularly useful for local integrations and command-line tools
- **Streamable HTTP**: this replaces the HTTP+SSE transport from previous protocol version. In the Streamable HTTP transport, the server operates as an independent process that can handle multiple client connections. This transport uses HTTP POST and GET requests. Server can optionally make use of Server-Sent Events (SSE) to stream multiple server messages. This permits basic MCP servers, as well as more feature-rich servers supporting streaming and server-to-client notifications and requests.
- **SSE**: legacy transport for HTTP-based MCP servers that use Server-Sent Events transport  

Overall, you should use stdio for building command-line tools, implementing local integrations and working with shell scripts. You should use Streamable HTTP transports when you need a flexible and efficient way for AI agents to communicate with tools and services, especially when dealing with stateless communication or when minimizing resource usage is crucial.

You can also use **custom transports** implementation for your specific needs. 


Let's now connect to the MCP server using stdio transport. First of all, we will use the class `MCPClient` to connect to the [AWS Documentation MCP Server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/). This server provides tools to access AWS documentation, search for content, and get recommendations.

In [3]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `stdio_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [4]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent("What is Amazon Bedrock pricing model. Be concise.")

I'll look up the Amazon Bedrock pricing model for you. Let me search the AWS documentation to get this information.
Tool #1: search_documentation
Now let me get the detailed pricing information from the main pricing documentation page:
Tool #2: read_documentation
I'll check more about the Provisioned Throughput option mentioned:
Tool #3: read_documentation
Based on the AWS documentation, here's a concise explanation of Amazon Bedrock's pricing model:

# Amazon Bedrock Pricing Model

Amazon Bedrock offers two primary pricing models:

1. **On-demand Usage**: 
   - Pay-as-you-go based on the volume of input and output tokens processed
   - Different pricing for each foundation model
   - No upfront commitments

2. **Provisioned Throughput**:
   - Fixed cost for dedicated capacity
   - Charged hourly based on:
     - Model selected
     - Number of Model Units (MUs) purchased
     - Commitment duration (no commitment, 1-month, or 6-month terms)
   - Required for custom models
   - Longer c

### Connect to MCP server using Streamable HTTP

Let's now connect to the MCP server using Streamable HTTP transport. First let's start a simple MCP server using Streamable HTTP transport. 

For this example we will create our own MCP server. The architecture will look as following

<div style="text-align:center">
    <img src="images/architecture_2.png" width="65%" />
</div>

In [5]:
# Create an MCP server
mcp = FastMCP("Calculator Server")

# Define a tool


@mcp.tool(description="Calculator tool which performs calculations")
def calculator(x: int, y: int) -> int:
    return x + y


@mcp.tool(description="This is a long running tool")
def long_running_tool(name: str) -> str:
    time.sleep(25)
    return f"Hello {name}"


def main():
    mcp.run(transport="streamable-http", mount_path="mcp")

Let's now start a thread with the `streamable-http` server

In [ ]:
thread = threading.Thread(target=main)
thread.start()

INFO:     Started server process [146618]
INFO:     Waiting for application startup.


[09/04/25 02:06:11] INFO     StreamableHTTP session manager started                  ]8;id=3767;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=108636;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#110\110]8;;\

INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:41992 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42006 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42004 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:42018 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42020 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42036 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42042 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42058 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42046 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:42066 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42078 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42084 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:39756 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:39758 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:39768 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:39778 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:49454 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:49456 - "

#### Integrating Streamable HTTP client with Agent

Now let's use `streamablehttp_client` integrate this server with a simple agent. 

In [7]:
def create_streamable_http_transport():
    return streamablehttp_client("http://localhost:8000/mcp")


streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `streamable_http_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [8]:
with streamable_http_mcp_client:
    tools = streamable_http_mcp_client.list_tools_sync()

    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = str(agent("What is 2 + 2?"))

[09/04/25 02:06:28] INFO     Created new transport with session ID:                  ]8;id=786412;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=507830;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             cce608b8a4894bd98ed63845f73d6959                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=933702;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=286149;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: cce608b8a4894bd98ed63845f73d6959           ]8;id=646649;file:///home/participant/.local/lib/python3.10/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=739673;file:///home/participant/.local/lib/python3.10/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=165271;file:///home/participant/.local/lib/python3.10/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=650004;file:///home/participant/.local/lib/python3.10/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: GET http://localhost:8000/mcp "HTTP/1.1 200 OK"          ]8;id=690974;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=915153;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

...

[09/04/25 02:06:30] INFO     Processing request of type CallToolRequest                               ]8;id=64225;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=642819;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/lowlevel/server.py#624\624]8;;\

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=852495;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=499637;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

The result of 2 + 2 is 4.

[09/04/25 02:06:31] INFO     Terminating session: cce608b8a4894bd98ed63845f73d6959           ]8;id=941072;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=303564;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=16869;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=195014;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

### Direct Tool Invocation

While tools are typically invoked by the agent based on user requests, you can also call MCP tools directly. This can be useful for workflow scenarios where you orchestrate multiple tools together.

In [9]:
query = {"x": 10, "y": 20}

with streamable_http_mcp_client:
    # direct tool invocation
    result = streamable_http_mcp_client.call_tool_sync(
        tool_use_id="tool-123", name="calculator", arguments=query
    )

    # Process the result
    print(f"Calculation result: {result['content'][0]['text']}")

[09/04/25 02:06:37] INFO     Created new transport with session ID:                  ]8;id=645775;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=936887;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             8d65fcd2bb8040898eb06247650495f3                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=868356;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=949408;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 8d65fcd2bb8040898eb06247650495f3           ]8;id=645617;file:///home/participant/.local/lib/python3.10/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=512275;file:///home/participant/.local/lib/python3.10/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=283571;file:///home/participant/.local/lib/python3.10/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=685194;file:///home/participant/.local/lib/python3.10/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: GET http://localhost:8000/mcp "HTTP/1.1 200 OK"          ]8;id=837118;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=601009;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

...

                    INFO     Processing request of type ListToolsRequest                              ]8;id=520920;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=719421;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/lowlevel/server.py#624\624]8;;\

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=852801;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=992707;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

Calculation result: 30


                    INFO     Terminating session: 8d65fcd2bb8040898eb06247650495f3           ]8;id=890001;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=514957;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=614356;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=770785;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

You can optionally also provide `read_timeout_seconds` while calling an MCP server tool to avoid it running for too long

In [10]:
with streamable_http_mcp_client:
    try:
        result = streamable_http_mcp_client.call_tool_sync(
            tool_use_id="tool-123",
            name="long_running_tool",
            arguments={"name": "Amazon"},
            read_timeout_seconds=timedelta(seconds=30),
        )

        if result["status"] == "error":
            print(f"Tool execution failed: {result['content'][0]['text']}")
        else:
            print(f"Tool execution succeeded: {result['content'][0]['text']}")
    except Exception as e:
        print(f"Tool call timed out or failed: {str(e)}")

[09/04/25 02:06:42] INFO     Created new transport with session ID:                  ]8;id=461883;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=806605;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             430236b58c3c41008a19fde5e81c8b18                                                      

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=385741;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=951147;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 430236b58c3c41008a19fde5e81c8b18           ]8;id=658954;file:///home/participant/.local/lib/python3.10/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=947519;file:///home/participant/.local/lib/python3.10/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=96751;file:///home/participant/.local/lib/python3.10/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=225895;file:///home/participant/.local/lib/python3.10/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: GET http://localhost:8000/mcp "HTTP/1.1 200 OK"          ]8;id=157953;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=357000;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

...

[09/04/25 02:07:07] INFO     Processing request of type ListToolsRequest                              ]8;id=153639;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=768511;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/lowlevel/server.py#624\624]8;;\

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=553585;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=20856;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

Tool execution succeeded: Hello Amazon


                    INFO     Terminating session: 430236b58c3c41008a19fde5e81c8b18           ]8;id=256200;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=368806;file:///home/participant/.local/lib/python3.10/site-packages/mcp/server/streamable_http.py#630\630]8;;\

                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=629178;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=895956;file:///home/participant/.local/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

### Interacting with multiple MCP servers

With Strands Agents you can also interact with multiple MCP servers using the same agent and configure tools setups such as the max number of tools that can be used in parallel (`max_parallel_tools`). Let's create a new agent to showcase this configuration:

<div style="text-align:center">
    <img src="images/architecture_3.png" width="85%" />
</div>

In this agent, we will again use the AWS Documentation MCP server and we will also use the [AWS CDK MCP Server](https://awslabs.github.io/mcp/servers/cdk-mcp-server/) which helps with AWS Cloud Development Kit (CDK) best practices, infrastructure as code patterns and security compliance with CDK Nag.

First let's connect to the two MCP servers using the stdio transport

In [11]:
# Connect to an MCP server using stdio transport
aws_docs_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Connect to an MCP server using stdio transport
cdk_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(command="uvx", args=["awslabs.cdk-mcp-server@latest"])
    )
)

#### Create Agent with MCP servers

Next we will create the agent with the tools from both MCP servers

In [12]:
# Create an agent with MCP tools
with aws_docs_mcp_client, cdk_mcp_client:
    # Get the tools from the MCP server
    tools = aws_docs_mcp_client.list_tools_sync() + cdk_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent(
        "What is Amazon Bedrock pricing model. Be concise. Also what are the best practices related to CDK?"
    )

[09/04/25 02:07:42] INFO     Found credentials from IAM Role:                                   ]8;id=710378;file:///home/participant/.local/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=620907;file:///home/participant/.local/lib/python3.10/site-packages/botocore/credentials.py#1138\1138]8;;\
                             CodeEditorV2-CodeEditorInstanceBootstrapRole-etk4tqcPN52E                             

I'll provide you with information about Amazon Bedrock's pricing model and CDK best practices. Let me search for the relevant documentation.

First, let me check Amazon Bedrock pricing:
Tool #1: search_documentation
Now, let me get the detailed information about Amazon Bedrock pricing:
Tool #2: read_documentation
Now, let me check for CDK best practices:
Tool #3: CDKGeneralGuidance
# Amazon Bedrock Pricing Model and CDK Best Practices

## Amazon Bedrock Pricing

Amazon Bedrock uses a pay-as-you-go pricing model based on:

1. **Token-based pricing**: You pay for the volume of input tokens and output tokens processed.
2. **Model-specific rates**: Each foundation model has its own pricing structure.
3. **Provisioned Throughput option**: You can purchase dedicated capacity for models at potentially lower rates than on-demand usage.

Pricing varies by model provider and model version. For detailed pricing information, you can visit the [Amazon Bedrock pricing page](https://aws.amazon.com/be

### Congratulations!

In this notebook you learned how to connect with MCP servers using Strands Agent and two MCP transport protocols: stdio and Streamable HTTP. You also learned how to connect multiple MCP servers to the same agent. Next, let's see how to use different models with your agent